# Advanced EDA Pipeline - Продвинутый анализ данных

Комплексный пайплайн для:
- Автоматический анализ данных
- Продвинутые визуализации
- Feature importance и корреляции
- Выявление аномалий
- Distribution analysis
- Interactive plots

In [ ]:
!pip install pandas numpy matplotlib seaborn plotly scikit-learn scipy pandas-profiling sweetviz autoviz phik missingno -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Статистика и ML
from scipy import stats
from scipy.stats import normaltest, shapiro, chi2_contingency
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

# Дополнительные библиотеки для EDA
import missingno as msno
from phik import phik_matrix

# Настройки визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Настройки pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("✓ Библиотеки загружены!")

## 1. Загрузка данных

In [ ]:
# === ВАШИ ДАННЫЕ ===
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

TARGET_COL = 'target'  # Укажите целевую переменную
ID_COL = 'id'  # Укажите ID колонку

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nMemory usage: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 2. Базовый обзор данных

In [ ]:
def dataset_overview(df, name='Dataset'):
    """
    Полный обзор датасета
    """
    print("="*80)
    print(f"📊 {name} OVERVIEW")
    print("="*80)
    
    # Размер
    print(f"\n📐 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"💾 Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Типы данных
    print(f"\n📋 Data Types:")
    dtype_counts = df.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"  {dtype}: {count} columns")
    
    # Категории колонок
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    datetime_cols = df.select_dtypes(include=['datetime64']).columns.tolist()
    
    print(f"\n🔢 Numeric columns: {len(numeric_cols)}")
    print(f"📝 Categorical columns: {len(categorical_cols)}")
    print(f"📅 Datetime columns: {len(datetime_cols)}")
    
    # Пропуски
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing': missing,
        'Percent': missing_pct
    })
    missing_df = missing_df[missing_df['Missing'] > 0].sort_values('Percent', ascending=False)
    
    if len(missing_df) > 0:
        print(f"\n❌ Missing Values: {len(missing_df)} columns")
        print(missing_df.head(10))
    else:
        print("\n✅ No missing values!")
    
    # Дубликаты
    duplicates = df.duplicated().sum()
    print(f"\n🔄 Duplicate rows: {duplicates:,} ({duplicates/len(df)*100:.2f}%)")
    
    # Уникальные значения
    print(f"\n🎯 Unique values per column:")
    unique_counts = df.nunique().sort_values()
    print(unique_counts.head(10))
    
    print("\n" + "="*80)

# Применение
dataset_overview(train_df, 'TRAIN')
dataset_overview(test_df, 'TEST')

In [ ]:
# Первые и последние строки
print("\n📌 First 5 rows:")
display(train_df.head())

print("\n📌 Last 5 rows:")
display(train_df.tail())

# Случайные строки
print("\n📌 Random 5 rows:")
display(train_df.sample(5))

In [ ]:
# Детальная статистика
print("\n📊 Descriptive Statistics:")
display(train_df.describe(include='all').T)

## 3. Анализ пропусков

In [ ]:
# Визуализация пропусков с missingno
fig, axes = plt.subplots(2, 2, figsize=(20, 12))

# Matrix
plt.subplot(2, 2, 1)
msno.matrix(train_df, ax=plt.gca(), fontsize=10)
plt.title('Missing Data Matrix', fontsize=14, fontweight='bold')

# Bar chart
plt.subplot(2, 2, 2)
msno.bar(train_df, ax=plt.gca(), fontsize=10)
plt.title('Missing Data Bar Chart', fontsize=14, fontweight='bold')

# Heatmap
plt.subplot(2, 2, 3)
msno.heatmap(train_df, ax=plt.gca(), fontsize=10)
plt.title('Missing Data Correlation Heatmap', fontsize=14, fontweight='bold')

# Dendrogram
plt.subplot(2, 2, 4)
msno.dendrogram(train_df, ax=plt.gca(), fontsize=10)
plt.title('Missing Data Dendrogram', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Детальный анализ пропусков
def analyze_missing_values(df):
    """
    Детальный анализ пропущенных значений
    """
    missing_stats = []
    
    for col in df.columns:
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            missing_pct = (missing_count / len(df)) * 100
            dtype = df[col].dtype
            unique_count = df[col].nunique()
            
            missing_stats.append({
                'Column': col,
                'Missing_Count': missing_count,
                'Missing_Percent': missing_pct,
                'Dtype': dtype,
                'Unique_Values': unique_count
            })
    
    if missing_stats:
        missing_df = pd.DataFrame(missing_stats)
        missing_df = missing_df.sort_values('Missing_Percent', ascending=False)
        
        print("\n📊 MISSING VALUES ANALYSIS:")
        print("="*80)
        display(missing_df)
        
        # Визуализация
        fig = px.bar(
            missing_df,
            x='Column',
            y='Missing_Percent',
            title='Missing Values Percentage by Column',
            labels={'Missing_Percent': 'Missing %'},
            color='Missing_Percent',
            color_continuous_scale='Reds'
        )
        fig.update_layout(height=500)
        fig.show()
        
        return missing_df
    else:
        print("✅ No missing values found!")
        return None

missing_analysis = analyze_missing_values(train_df)

## 4. Анализ целевой переменной

In [ ]:
if TARGET_COL in train_df.columns:
    print(f"\n🎯 TARGET VARIABLE ANALYSIS: '{TARGET_COL}'")
    print("="*80)
    
    # Проверка типа задачи
    n_unique = train_df[TARGET_COL].nunique()
    is_classification = n_unique < 20
    
    if is_classification:
        print(f"\n📊 Task Type: CLASSIFICATION ({n_unique} classes)")
        
        # Распределение классов
        target_counts = train_df[TARGET_COL].value_counts()
        target_pcts = train_df[TARGET_COL].value_counts(normalize=True) * 100
        
        target_stats = pd.DataFrame({
            'Count': target_counts,
            'Percentage': target_pcts
        })
        
        print("\nClass Distribution:")
        display(target_stats)
        
        # Визуализация
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Class Distribution', 'Class Percentages'),
            specs=[[{'type': 'bar'}, {'type': 'pie'}]]
        )
        
        # Bar chart
        fig.add_trace(
            go.Bar(x=target_counts.index, y=target_counts.values, name='Count'),
            row=1, col=1
        )
        
        # Pie chart
        fig.add_trace(
            go.Pie(labels=target_counts.index, values=target_counts.values),
            row=1, col=2
        )
        
        fig.update_layout(height=400, showlegend=False)
        fig.show()
        
        # Проверка дисбаланса
        imbalance_ratio = target_counts.max() / target_counts.min()
        print(f"\n⚖️ Imbalance Ratio: {imbalance_ratio:.2f}")
        if imbalance_ratio > 5:
            print("⚠️ WARNING: Significant class imbalance detected!")
    
    else:
        print(f"\n📊 Task Type: REGRESSION")
        
        # Статистика
        print("\nTarget Statistics:")
        print(train_df[TARGET_COL].describe())
        
        # Визуализация распределения
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Histogram
        axes[0, 0].hist(train_df[TARGET_COL].dropna(), bins=50, edgecolor='black')
        axes[0, 0].set_title('Distribution', fontweight='bold')
        axes[0, 0].set_xlabel(TARGET_COL)
        axes[0, 0].set_ylabel('Frequency')
        
        # Box plot
        axes[0, 1].boxplot(train_df[TARGET_COL].dropna())
        axes[0, 1].set_title('Box Plot', fontweight='bold')
        axes[0, 1].set_ylabel(TARGET_COL)
        
        # Q-Q plot
        stats.probplot(train_df[TARGET_COL].dropna(), dist="norm", plot=axes[1, 0])
        axes[1, 0].set_title('Q-Q Plot', fontweight='bold')
        
        # KDE plot
        train_df[TARGET_COL].dropna().plot(kind='kde', ax=axes[1, 1])
        axes[1, 1].set_title('Kernel Density Estimate', fontweight='bold')
        axes[1, 1].set_xlabel(TARGET_COL)
        
        plt.tight_layout()
        plt.show()
        
        # Проверка нормальности
        statistic, p_value = normaltest(train_df[TARGET_COL].dropna())
        print(f"\n📊 Normality Test (D'Agostino-Pearson):")
        print(f"  Statistic: {statistic:.4f}")
        print(f"  P-value: {p_value:.4f}")
        if p_value < 0.05:
            print("  ❌ Target is NOT normally distributed")
        else:
            print("  ✅ Target appears normally distributed")

print("\n" + "="*80)

## 5. Анализ числовых признаков

In [ ]:
# Получение числовых колонок
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
if ID_COL in numeric_cols:
    numeric_cols.remove(ID_COL)
if TARGET_COL in numeric_cols:
    numeric_cols.remove(TARGET_COL)

print(f"\n🔢 NUMERIC FEATURES ANALYSIS ({len(numeric_cols)} features)")
print("="*80)

if len(numeric_cols) > 0:
    # Детальная статистика
    numeric_stats = train_df[numeric_cols].describe().T
    numeric_stats['missing'] = train_df[numeric_cols].isnull().sum()
    numeric_stats['missing_pct'] = (numeric_stats['missing'] / len(train_df)) * 100
    numeric_stats['unique'] = train_df[numeric_cols].nunique()
    numeric_stats['skew'] = train_df[numeric_cols].skew()
    numeric_stats['kurtosis'] = train_df[numeric_cols].kurtosis()
    
    print("\nNumeric Features Statistics:")
    display(numeric_stats)
    
    # Выявление сильно скошенных признаков
    high_skew = numeric_stats[abs(numeric_stats['skew']) > 1].sort_values('skew', key=abs, ascending=False)
    if len(high_skew) > 0:
        print(f"\n⚠️ Highly Skewed Features (|skew| > 1): {len(high_skew)}")
        print(high_skew[['mean', 'std', 'skew']])

In [ ]:
# Распределения числовых признаков
if len(numeric_cols) > 0:
    n_cols = min(4, len(numeric_cols))
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
    
    for idx, col in enumerate(numeric_cols[:20]):  # Первые 20 для примера
        ax = axes[idx]
        train_df[col].hist(bins=50, ax=ax, edgecolor='black', alpha=0.7)
        ax.set_title(f'{col}\nSkew: {train_df[col].skew():.2f}', fontweight='bold')
        ax.set_xlabel(col)
        ax.set_ylabel('Frequency')
    
    # Скрыть лишние subplots
    for idx in range(len(numeric_cols), len(axes)):
        axes[idx].set_visible(False)
    
    plt.suptitle('Numeric Features Distributions', fontsize=16, fontweight='bold', y=1.0)
    plt.tight_layout()
    plt.show()

In [ ]:
# Box plots для выявления выбросов
if len(numeric_cols) > 0:
    fig = make_subplots(
        rows=(len(numeric_cols[:12]) + 3) // 4,
        cols=4,
        subplot_titles=numeric_cols[:12]
    )
    
    for idx, col in enumerate(numeric_cols[:12]):
        row = idx // 4 + 1
        col_num = idx % 4 + 1
        
        fig.add_trace(
            go.Box(y=train_df[col].dropna(), name=col, showlegend=False),
            row=row, col=col_num
        )
    
    fig.update_layout(height=300 * ((len(numeric_cols[:12]) + 3) // 4), 
                      title_text="Box Plots - Outlier Detection")
    fig.show()

## 6. Корреляционный анализ

In [ ]:
if len(numeric_cols) > 0:
    print("\n🔗 CORRELATION ANALYSIS")
    print("="*80)
    
    # Pearson correlation
    corr_matrix = train_df[numeric_cols].corr()
    
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    
    # Heatmap
    sns.heatmap(corr_matrix, annot=True if len(numeric_cols) < 15 else False, 
                cmap='coolwarm', center=0, square=True, ax=axes[0],
                fmt='.2f', linewidths=0.5)
    axes[0].set_title('Pearson Correlation Heatmap', fontsize=14, fontweight='bold')
    
    # Clustermap alternative
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True if len(numeric_cols) < 15 else False,
                cmap='coolwarm', center=0, square=True, ax=axes[1],
                fmt='.2f', linewidths=0.5)
    axes[1].set_title('Correlation Heatmap (Lower Triangle)', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Топ корреляций
    corr_pairs = corr_matrix.unstack()
    corr_pairs = corr_pairs[corr_pairs != 1]
    high_corr = corr_pairs[abs(corr_pairs) > 0.7].sort_values(key=abs, ascending=False)
    
    if len(high_corr) > 0:
        print(f"\n⚠️ High Correlations (|r| > 0.7):")
        print(high_corr.head(20))
    
    # Корреляция с таргетом
    if TARGET_COL in train_df.columns and TARGET_COL not in numeric_cols:
        # Для классификации - используем point-biserial или mutual information
        if train_df[TARGET_COL].nunique() < 20:
            target_corr = []
            for col in numeric_cols:
                corr, _ = stats.pointbiserialr(train_df[TARGET_COL], train_df[col])
                target_corr.append((col, corr))
            target_corr = pd.Series(dict(target_corr)).sort_values(key=abs, ascending=False)
        else:
            target_corr = train_df[numeric_cols + [TARGET_COL]].corr()[TARGET_COL].drop(TARGET_COL)
            target_corr = target_corr.sort_values(key=abs, ascending=False)
        
        print(f"\n🎯 Top 15 Features Correlated with Target:")
        print(target_corr.head(15))
        
        # Визуализация
        fig = px.bar(
            x=target_corr.head(15).values,
            y=target_corr.head(15).index,
            orientation='h',
            title='Top 15 Features by Correlation with Target',
            labels={'x': 'Correlation', 'y': 'Feature'},
            color=target_corr.head(15).values,
            color_continuous_scale='RdBu'
        )
        fig.update_layout(height=500)
        fig.show()

In [ ]:
# Phi-K correlation для mixed data types
try:
    print("\n📊 Phi-K Correlation Matrix (works with categorical data):")
    phik_corr = train_df[numeric_cols[:15]].phik_matrix()  # Первые 15 для скорости
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(phik_corr, annot=True, cmap='viridis', center=0.5, square=True, fmt='.2f')
    plt.title('Phi-K Correlation Matrix', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Phi-K calculation skipped: {e}")

## 7. Анализ категориальных признаков

In [ ]:
categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\n📝 CATEGORICAL FEATURES ANALYSIS ({len(categorical_cols)} features)")
print("="*80)

if len(categorical_cols) > 0:
    # Статистика по категориальным признакам
    cat_stats = []
    for col in categorical_cols:
        cat_stats.append({
            'Feature': col,
            'Unique_Values': train_df[col].nunique(),
            'Missing': train_df[col].isnull().sum(),
            'Missing_Pct': (train_df[col].isnull().sum() / len(train_df)) * 100,
            'Most_Common': train_df[col].mode()[0] if len(train_df[col].mode()) > 0 else None,
            'Most_Common_Freq': train_df[col].value_counts().iloc[0] if len(train_df[col].value_counts()) > 0 else 0
        })
    
    cat_stats_df = pd.DataFrame(cat_stats).sort_values('Unique_Values', ascending=False)
    print("\nCategorical Features Statistics:")
    display(cat_stats_df)

In [ ]:
# Визуализация топ категориальных признаков
if len(categorical_cols) > 0:
    for col in categorical_cols[:6]:  # Первые 6
        value_counts = train_df[col].value_counts().head(15)
        
        fig = px.bar(
            x=value_counts.index,
            y=value_counts.values,
            title=f"Distribution of '{col}' (Top 15)",
            labels={'x': col, 'y': 'Count'},
            color=value_counts.values,
            color_continuous_scale='Blues'
        )
        fig.update_layout(height=400, showlegend=False)
        fig.show()

In [ ]:
# Категориальные признаки vs Target
if TARGET_COL in train_df.columns and len(categorical_cols) > 0:
    print(f"\n📊 Categorical Features vs Target:")
    
    for col in categorical_cols[:5]:  # Первые 5
        if train_df[TARGET_COL].nunique() < 20:  # Классификация
            ct = pd.crosstab(train_df[col], train_df[TARGET_COL], normalize='index') * 100
            
            fig = px.bar(
                ct.head(15),
                title=f"'{col}' vs Target (Top 15 categories)",
                labels={'value': 'Percentage', 'variable': 'Target'},
                barmode='group'
            )
            fig.update_layout(height=500)
            fig.show()
            
            # Chi-square test
            ct_counts = pd.crosstab(train_df[col], train_df[TARGET_COL])
            chi2, p_value, dof, expected = chi2_contingency(ct_counts)
            print(f"\n  {col} - Chi-square test:")
            print(f"    Chi2 statistic: {chi2:.4f}")
            print(f"    P-value: {p_value:.4f}")
            if p_value < 0.05:
                print(f"    ✅ Significant relationship with target")
            else:
                print(f"    ❌ No significant relationship with target")

## 8. Feature Importance

In [ ]:
if TARGET_COL in train_df.columns and len(numeric_cols) > 0:
    print("\n🏆 FEATURE IMPORTANCE ANALYSIS")
    print("="*80)
    
    # Подготовка данных
    X = train_df[numeric_cols].fillna(train_df[numeric_cols].median())
    y = train_df[TARGET_COL]
    
    # Random Forest Feature Importance
    if y.nunique() < 20:  # Классификация
        rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    else:  # Регрессия
        from sklearn.ensemble import RandomForestRegressor
        rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    
    rf.fit(X, y)
    
    feature_importance = pd.DataFrame({
        'Feature': numeric_cols,
        'Importance': rf.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\n📊 Top 20 Features by Random Forest Importance:")
    print(feature_importance.head(20))
    
    # Визуализация
    fig = px.bar(
        feature_importance.head(20),
        x='Importance',
        y='Feature',
        orientation='h',
        title='Top 20 Features - Random Forest Importance',
        color='Importance',
        color_continuous_scale='Viridis'
    )
    fig.update_layout(height=600, yaxis={'categoryorder': 'total ascending'})
    fig.show()

In [ ]:
# Mutual Information
if TARGET_COL in train_df.columns and len(numeric_cols) > 0:
    print("\n🔗 Mutual Information Scores:")
    
    X = train_df[numeric_cols].fillna(train_df[numeric_cols].median())
    y = train_df[TARGET_COL]
    
    if y.nunique() < 20:
        mi_scores = mutual_info_classif(X, y, random_state=42)
    else:
        mi_scores = mutual_info_regression(X, y, random_state=42)
    
    mi_df = pd.DataFrame({
        'Feature': numeric_cols,
        'MI_Score': mi_scores
    }).sort_values('MI_Score', ascending=False)
    
    print(mi_df.head(20))
    
    # Визуализация
    fig = px.bar(
        mi_df.head(20),
        x='MI_Score',
        y='Feature',
        orientation='h',
        title='Top 20 Features - Mutual Information',
        color='MI_Score',
        color_continuous_scale='Plasma'
    )
    fig.update_layout(height=600, yaxis={'categoryorder': 'total ascending'})
    fig.show()

## 9. Outlier Detection

In [ ]:
if len(numeric_cols) > 0:
    print("\n🔍 OUTLIER DETECTION")
    print("="*80)
    
    # IQR method
    outlier_stats = []
    for col in numeric_cols:
        Q1 = train_df[col].quantile(0.25)
        Q3 = train_df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = train_df[(train_df[col] < lower_bound) | (train_df[col] > upper_bound)][col]
        outlier_count = len(outliers)
        outlier_pct = (outlier_count / len(train_df)) * 100
        
        outlier_stats.append({
            'Feature': col,
            'Outliers': outlier_count,
            'Outlier_Pct': outlier_pct,
            'Lower_Bound': lower_bound,
            'Upper_Bound': upper_bound
        })
    
    outlier_df = pd.DataFrame(outlier_stats).sort_values('Outlier_Pct', ascending=False)
    outlier_df = outlier_df[outlier_df['Outliers'] > 0]
    
    if len(outlier_df) > 0:
        print("\nFeatures with Outliers (IQR method):")
        display(outlier_df)
        
        # Визуализация
        fig = px.bar(
            outlier_df.head(15),
            x='Feature',
            y='Outlier_Pct',
            title='Top 15 Features by Outlier Percentage',
            labels={'Outlier_Pct': 'Outlier %'},
            color='Outlier_Pct',
            color_continuous_scale='Reds'
        )
        fig.update_layout(height=500)
        fig.show()
    else:
        print("✅ No outliers detected using IQR method")

In [ ]:
# Isolation Forest для аномалий
if len(numeric_cols) > 0:
    print("\n🌲 Isolation Forest Anomaly Detection:")
    
    X = train_df[numeric_cols].fillna(train_df[numeric_cols].median())
    
    iso_forest = IsolationForest(contamination=0.05, random_state=42, n_jobs=-1)
    anomaly_labels = iso_forest.fit_predict(X)
    
    n_anomalies = (anomaly_labels == -1).sum()
    anomaly_pct = (n_anomalies / len(train_df)) * 100
    
    print(f"  Detected anomalies: {n_anomalies} ({anomaly_pct:.2f}%)")
    
    train_df['is_anomaly'] = anomaly_labels
    
    # Визуализация с PCA
    if len(numeric_cols) >= 2:
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X)
        
        fig = px.scatter(
            x=X_pca[:, 0],
            y=X_pca[:, 1],
            color=anomaly_labels,
            title='Anomaly Detection (PCA projection)',
            labels={'x': 'PC1', 'y': 'PC2', 'color': 'Anomaly'},
            color_discrete_map={1: 'blue', -1: 'red'}
        )
        fig.update_layout(height=600)
        fig.show()

## 10. Dimensionality Reduction Visualization

In [ ]:
if len(numeric_cols) > 2 and TARGET_COL in train_df.columns:
    print("\n🎨 DIMENSIONALITY REDUCTION VISUALIZATION")
    print("="*80)
    
    X = train_df[numeric_cols].fillna(train_df[numeric_cols].median())
    y = train_df[TARGET_COL]
    
    # Стандартизация
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # PCA
    print("\nComputing PCA...")
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    print(f"PCA Explained Variance: {pca.explained_variance_ratio_.sum():.2%}")
    
    # t-SNE
    print("\nComputing t-SNE (может занять время)...")
    tsne = TSNE(n_components=2, random_state=42, n_jobs=-1)
    X_tsne = tsne.fit_transform(X_scaled[:1000])  # Первые 1000 для скорости
    
    # Визуализация
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('PCA', 't-SNE'),
        specs=[[{'type': 'scatter'}, {'type': 'scatter'}]]
    )
    
    # PCA plot
    for target_val in y.unique()[:10]:  # Максимум 10 классов
        mask = y == target_val
        fig.add_trace(
            go.Scatter(
                x=X_pca[mask, 0],
                y=X_pca[mask, 1],
                mode='markers',
                name=str(target_val),
                marker=dict(size=5, opacity=0.6)
            ),
            row=1, col=1
        )
    
    # t-SNE plot
    for target_val in y[:1000].unique()[:10]:
        mask = y[:1000] == target_val
        fig.add_trace(
            go.Scatter(
                x=X_tsne[mask, 0],
                y=X_tsne[mask, 1],
                mode='markers',
                name=str(target_val),
                marker=dict(size=5, opacity=0.6),
                showlegend=False
            ),
            row=1, col=2
        )
    
    fig.update_xaxes(title_text="PC1", row=1, col=1)
    fig.update_yaxes(title_text="PC2", row=1, col=1)
    fig.update_xaxes(title_text="Dimension 1", row=1, col=2)
    fig.update_yaxes(title_text="Dimension 2", row=1, col=2)
    
    fig.update_layout(height=500, title_text="Dimensionality Reduction Visualization")
    fig.show()

## 11. Train vs Test Distribution Comparison

In [ ]:
print("\n🔄 TRAIN vs TEST DISTRIBUTION COMPARISON")
print("="*80)

# Сравнение для числовых признаков
common_numeric = [col for col in numeric_cols if col in test_df.columns]

if len(common_numeric) > 0:
    for col in common_numeric[:6]:  # Первые 6
        fig = go.Figure()
        
        fig.add_trace(go.Histogram(
            x=train_df[col].dropna(),
            name='Train',
            opacity=0.7,
            nbinsx=50
        ))
        
        fig.add_trace(go.Histogram(
            x=test_df[col].dropna(),
            name='Test',
            opacity=0.7,
            nbinsx=50
        ))
        
        fig.update_layout(
            title=f"Distribution Comparison: '{col}'",
            xaxis_title=col,
            yaxis_title='Frequency',
            barmode='overlay',
            height=400
        )
        fig.show()
        
        # KS test
        ks_stat, ks_pval = stats.ks_2samp(
            train_df[col].dropna(),
            test_df[col].dropna()
        )
        print(f"\n{col} - Kolmogorov-Smirnov Test:")
        print(f"  Statistic: {ks_stat:.4f}")
        print(f"  P-value: {ks_pval:.4f}")
        if ks_pval < 0.05:
            print(f"  ⚠️ WARNING: Distributions are significantly different!")
        else:
            print(f"  ✅ Distributions are similar")

## 12. Automated Report Generation

In [ ]:
# Pandas Profiling для автоматического отчета
try:
    from ydata_profiling import ProfileReport
    
    print("\n📋 Generating Automated EDA Report...")
    print("⏳ This may take a few minutes...")
    
    profile = ProfileReport(
        train_df,
        title='Automated EDA Report',
        explorative=True,
        minimal=False
    )
    
    profile.to_file('eda_report.html')
    print("\n✅ Report saved as 'eda_report.html'")
    
except ImportError:
    print("\n⚠️ ydata-profiling not installed. Run: pip install ydata-profiling")
except Exception as e:
    print(f"\n⚠️ Error generating report: {e}")

## 13. Key Insights Summary

In [ ]:
print("\n" + "="*80)
print("📊 KEY INSIGHTS SUMMARY")
print("="*80)

print(f"\n1️⃣ Dataset Overview:")
print(f"   • Train size: {train_df.shape[0]:,} rows × {train_df.shape[1]} columns")
print(f"   • Test size: {test_df.shape[0]:,} rows × {test_df.shape[1]} columns")
print(f"   • Numeric features: {len(numeric_cols)}")
print(f"   • Categorical features: {len(categorical_cols)}")

if TARGET_COL in train_df.columns:
    print(f"\n2️⃣ Target Variable:")
    if train_df[TARGET_COL].nunique() < 20:
        print(f"   • Type: Classification ({train_df[TARGET_COL].nunique()} classes)")
        imbalance = train_df[TARGET_COL].value_counts().max() / train_df[TARGET_COL].value_counts().min()
        print(f"   • Imbalance ratio: {imbalance:.2f}")
    else:
        print(f"   • Type: Regression")
        print(f"   • Range: [{train_df[TARGET_COL].min():.2f}, {train_df[TARGET_COL].max():.2f}]")

if missing_analysis is not None and len(missing_analysis) > 0:
    print(f"\n3️⃣ Data Quality:")
    print(f"   • Features with missing values: {len(missing_analysis)}")
    print(f"   • Highest missing rate: {missing_analysis['Missing_Percent'].max():.1f}%")
else:
    print(f"\n3️⃣ Data Quality:")
    print(f"   • ✅ No missing values!")

print(f"   • Duplicate rows: {train_df.duplicated().sum():,}")

if len(numeric_cols) > 0:
    print(f"\n4️⃣ Feature Characteristics:")
    high_skew = abs(train_df[numeric_cols].skew()) > 1
    print(f"   • Highly skewed features: {high_skew.sum()}")
    
    if 'outlier_df' in locals() and len(outlier_df) > 0:
        print(f"   • Features with outliers: {len(outlier_df)}")

print("\n" + "="*80)
print("✅ EDA Complete!")
print("="*80)